# Vision Transformer Classification Task

- 기수: 13기

- 이름: 이진우

- 작성자: 13기 강승우, 조지성, 한연주

- Reference: 24-2 RNN+Transformer Assignment / 11기 김정우

본 과제에서는 간단한 Vision Transformer 모델을 Scratch 에서부터 구현해보는 내용을 담고 있습니다.

![image.png](attachment:image.png)

## 1. Setting

In [1]:
import numpy as np

from tqdm import tqdm, trange

import torch
import torch.nn as nn
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from torch.utils.data import DataLoader

from torchvision.transforms import ToTensor
from torchvision.datasets.mnist import MNIST

np.random.seed(2025)
torch.manual_seed(2025)

## 2. Training Format

아래 함수는 기본적인 학습 진행을 위한 템플릿입니다.

In [2]:
def main():
    # Loading data
    transform = ToTensor()

    train_set = MNIST(root='./datasets', train=True, download=True, transform=transform)
    test_set = MNIST(root='./datasets', train=False, download=True, transform=transform)

    train_loader = DataLoader(train_set, shuffle=True, batch_size=128)
    test_loader = DataLoader(test_set, shuffle=False, batch_size=128)

    # Defining model and training options
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Using device: ", device, f"({torch.cuda.get_device_name(device)})" if torch.cuda.is_available() else "")

    model = MyViT((1, 28, 28), n_patches=7, n_blocks=2, hidden_d=8, n_heads=2, out_d=10).to(device)
    N_EPOCHS = 30
    LR = 2e-3

    # Training loop
    optimizer = Adam(model.parameters(), lr=LR)
    criterion = CrossEntropyLoss()

    for epoch in trange(N_EPOCHS, desc="Training"):
        train_loss = 0.0
        for batch in tqdm(train_loader, desc=f"Epoch {epoch + 1} in training", leave=False):
            x, y = batch
            x, y = x.to(device), y.to(device)
            y_hat = model(x)
            loss = criterion(y_hat, y)

            train_loss += loss.detach().cpu().item() / len(train_loader)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f"Epoch {epoch + 1}/{N_EPOCHS} loss: {train_loss:.2f}")

    # Test loop
    with torch.no_grad():
        correct, total = 0, 0
        test_loss = 0.0

        for batch in tqdm(test_loader, desc="Testing"):
            x, y = batch
            x, y = x.to(device), y.to(device)
            y_hat = model(x)
            loss = criterion(y_hat, y)
            test_loss += loss.detach().cpu().item() / len(test_loader)

            correct += torch.sum(torch.argmax(y_hat, dim=1) == y).detach().cpu().item()
            total += len(x)

        print(f"Test loss: {test_loss:.2f}")
        print(f"Test accuracy: {correct / total * 100:.2f}%")

## 3. Forward Process

### 3-1. Patchifying and the linear mapping

기본적으로 하나의 MNIST 데이터는 (1, 28, 28) 으로 구성되어 있습니다.

본 과제에서는 7x7 (49개) 의 patch로 나누는 과정을 진행하게 됩니다.

In [3]:
def patchify(images, n_patches=7):
    n, c, h, w = images.shape
    assert h == w, "Patchify method is implemented for square images only"

    patch_size = h // n_patches  # 28//7 = 4
    num_patches = n_patches * n_patches
    patch_dim = c * patch_size * patch_size  # 1*4*4 = 16

    patches = torch.zeros(n, num_patches, patch_dim, device=images.device, dtype=images.dtype)

    for idx, image in enumerate(images):
        for i in range(n_patches):
            for j in range(n_patches):
                patch = image[:,
                              i*patch_size:(i+1)*patch_size,
                              j*patch_size:(j+1)*patch_size]
                patches[idx, i * n_patches + j] = patch.flatten()
    return patches

In [4]:
class MyViT(nn.Module):
  def __init__(self, chw=(1, 28, 28), n_patches=7):
    # Super constructor
    super(MyViT, self).__init__()

    # Attributes
    self.chw = chw # (C, H, W)
    self.n_patches = n_patches

    assert chw[1] % n_patches == 0, "Input shape not entirely divisible by number of patches"
    assert chw[2] % n_patches == 0, "Input shape not entirely divisible by number of patches"

  def forward(self, images):
    patches = patchify(images, self.n_patches)
    return patches

In [5]:
if __name__ == '__main__':
  # Current model
  model = MyViT(
    chw=(1, 28, 28),
    n_patches=7
  )

  x = torch.randn(7, 1, 28, 28) # Dummy images
  print(model(x).shape) # torch.Size([7, 49, 16])

torch.Size([7, 49, 16])


이어서 flatten patch에 간단한 linear mapping을 적용해 patch mapping을 진행하고자 합니다.

hidden dimmension은 8차원을 기본으로 하지만, 수정할 수 있습니다.

In [6]:
class MyViT(nn.Module):
  def __init__(self, chw=(1, 28, 28), n_patches=7, hidden_dim1=8):
    # Super constructor
    super(MyViT, self).__init__()

    # Attributes
    self.chw = chw # (C, H, W)
    self.n_patches = n_patches
    self.hidden_d = hidden_dim1

    assert chw[1] % n_patches == 0, "Input shape not entirely divisible by number of patches"
    assert chw[2] % n_patches == 0, "Input shape not entirely divisible by number of patches"

    self.patch_size = (chw[1] / n_patches, chw[2] / n_patches)

    self.input_d = int(chw[0] * self.patch_size[0] * self.patch_size[1])  # C * ph * pw
    self.linear_mapper = nn.Linear(self.input_d, self.hidden_d)

  def forward(self, images):
    patches = patchify(images, self.n_patches)
    tokens = self.linear_mapper(patches)

    return tokens

In [7]:
if __name__ == '__main__':
  # Current model
  model = MyViT(
    chw=(1, 28, 28),
    n_patches=7
  )

  x = torch.randn(7, 1, 28, 28) # Dummy images
  print(model(x).shape) # torch.Size([7, 49, 16])

torch.Size([7, 49, 8])


### 3-2. Adding the Classification Token

ViT에선 Transformer Encoder에 같이 v_class 토큰이 입력됩니다.

 (N, 29, 16)에 해당 토큰이 추가되어 (N, 50, 8) 으로 변환되게 됩니다.

In [8]:
class MyViT(nn.Module):
  def __init__(self, chw=(1, 28, 28), n_patches=7, hidden_dim1=8):
    super(MyViT, self).__init__()
    self.chw = chw
    self.n_patches = n_patches
    self.hidden_d = hidden_dim1

    assert chw[1] % n_patches == 0 and chw[2] % n_patches == 0
    self.patch_size = (chw[1] // n_patches, chw[2] // n_patches)

    # 1) Linear mapper
    self.input_d = int(chw[0] * self.patch_size[0] * self.patch_size[1])
    self.linear_mapper = nn.Linear(self.input_d, self.hidden_d)

    # 2) Learnable classification token (1, hidden_d)
    self.class_token = nn.Parameter(torch.rand(1, self.hidden_d))

  def forward(self, images):
    patches = patchify(images, self.n_patches)              # (N, 49, 16)
    tokens  = self.linear_mapper(patches)                   # (N, 49, hidden_d)
    # (N, 1, hidden_d) + (N, 49, hidden_d) -> (N, 50, hidden_d)
    tokens = torch.cat((self.class_token.expand(tokens.size(0), 1, -1), tokens), dim=1)
    return tokens

### 3-3. Positional Encoding

ViT에서의 Positional Encoding은 첫 번째 차원에는 고주파수 값을, 두 번째 차원에는 저주파수 값을 추가합니다. (구체적인 내용은 Attention is All you Need 논문을 참고해주세요!)

각 시퀀스에서 토큰 i에 대해 j 번째 좌표에 다음 값을 추가합니다:

![image.png](attachment:image.png)

In [9]:
def get_positional_embeddings(sequence_length, d):
    # PE(pos, 2i)   = sin(pos / 10000^{2i/d})
    # PE(pos, 2i+1) = cos(pos / 10000^{2i/d})
    result = torch.ones(sequence_length, d, dtype=torch.float32)
    for i in range(sequence_length):
        for j in range(d):
            angle = i / (10000 ** (j / d))
            if j % 2 == 0:
                result[i][j] = torch.sin(torch.tensor(angle))
            else:
                result[i][j] = torch.cos(torch.tensor(angle))
    return result

해당 부분까지 ViT Class에 더해주게 되면 아래와 같이 진행됩니다.

In [10]:
class MyViT(nn.Module):
  def __init__(self, chw=(1, 28, 28), n_patches=7, hidden_dim1=8):
    super(MyViT, self).__init__()
    self.chw = chw
    self.n_patches = n_patches
    self.hidden_d = hidden_dim1

    assert chw[1] % n_patches == 0 and chw[2] % n_patches == 0
    self.patch_size = (chw[1] // n_patches, chw[2] // n_patches)

    # 1) Linear mapper
    self.input_d = int(chw[0] * self.patch_size[0] * self.patch_size[1])
    self.linear_mapper = nn.Linear(self.input_d, self.hidden_d)

    # 2) Learnable class token
    self.class_token = nn.Parameter(torch.rand(1, self.hidden_d))

    # 3) Positional embedding (freeze)
    self.pos_embed = nn.Parameter(
        get_positional_embeddings(self.n_patches ** 2 + 1, self.hidden_d)
    )
    self.pos_embed.requires_grad = False

  def forward(self, images):
    patches = patchify(images, self.n_patches)              # (N, 49, 16)
    tokens  = self.linear_mapper(patches)                   # (N, 49, hidden_d)
    tokens  = torch.cat((self.class_token.expand(tokens.size(0), 1, -1), tokens), dim=1)  # (N, 50, hidden_d)

    pos = self.pos_embed.repeat(images.shape[0], 1, 1)     # (N, 50, hidden_d)
    out = tokens + pos
    return out


### 3-4. Multi-Head Self Attention

MSA를 위한 클래스는 아래와 같이 구현할 수 있습니다.

In [11]:
class MyMSA(nn.Module):
    def __init__(self, d, n_heads=2):
        super(MyMSA, self).__init__()
        self.d = d
        self.n_heads = n_heads
        assert d % n_heads == 0, f"Can't divide dimension {d} into {n_heads} heads"

        d_head = int(d / n_heads)
        self.q_mappings = nn.ModuleList([nn.Linear(d_head, d_head) for _ in range(self.n_heads)])
        self.k_mappings = nn.ModuleList([nn.Linear(d_head, d_head) for _ in range(self.n_heads)])
        self.v_mappings = nn.ModuleList([nn.Linear(d_head, d_head) for _ in range(self.n_heads)])
        self.d_head = d_head
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, sequences):
        # Sequences has shape (N, seq_length, token_dim)
        # We go into shape    (N, seq_length, n_heads, token_dim / n_heads)
        # And come back to    (N, seq_length, item_dim)  (through concatenation)
        result = []
        for sequence in sequences:
            seq_result = []
            for head in range(self.n_heads):
                q_mapping = self.q_mappings[head]
                k_mapping = self.k_mappings[head]
                v_mapping = self.v_mappings[head]

                seq = sequence[:, head * self.d_head: (head + 1) * self.d_head]
                q, k, v = q_mapping(seq), k_mapping(seq), v_mapping(seq)

                attention = self.softmax(q @ k.T / (self.d_head ** 0.5))
                seq_result.append(attention @ v)
            result.append(torch.hstack(seq_result))
        return torch.cat([torch.unsqueeze(r, dim=0) for r in result])

In [12]:
class MyViTBlock(nn.Module):
    def __init__(self, hidden_d, n_heads, mlp_ratio=4):
        super(MyViTBlock, self).__init__()
        self.hidden_d = hidden_d
        self.n_heads = n_heads

        self.norm1 = nn.LayerNorm(hidden_d)
        self.mhsa = MyMSA(hidden_d, n_heads)
        self.norm2 = nn.LayerNorm(hidden_d)
        self.mlp = nn.Sequential(
            nn.Linear(hidden_d, mlp_ratio * hidden_d),
            nn.GELU(),
            nn.Linear(mlp_ratio * hidden_d, hidden_d)
        )

    def forward(self, x):
        out = x + self.mhsa(self.norm1(x))
        out = out + self.mlp(self.norm2(out))
        return out

In [13]:
if __name__ == '__main__':
  model = MyViTBlock(hidden_d=8, n_heads=2)

  x = torch.randn(7, 50, 8)  # Dummy sequences
  print(model(x).shape)      # torch.Size([7, 50, 8])

torch.Size([7, 50, 8])


In [14]:
class MyViT(nn.Module):
  def __init__(self, chw=(1, 28, 28), n_patches=7, hidden_dim1=8):
    # Super constructor
    super(MyViT, self).__init__()

    # Attributes
    self.chw = chw # (C, H, W)
    self.n_patches = n_patches
    self.hidden_d = hidden_dim1

    assert chw[1] % n_patches == 0, "Input shape not entirely divisible by number of patches"
    assert chw[2] % n_patches == 0, "Input shape not entirely divisible by number of patches"
    self.patch_size = (chw[1] / n_patches, chw[2] / n_patches)

    # 1) Linear mapper
    self.input_d = int(chw[0] * self.patch_size[0] * self.patch_size[1])
    self.linear_mapper = nn.Linear(self.input_d, self.hidden_d)

    # 2) Learnable classifiation token
    self.class_token = nn.Parameter(torch.rand(1, self.hidden_d)) # Classification Token을 정의

    # 3) Positional embedding
    self.pos_embed = nn.Parameter(torch.tensor(get_positional_embeddings(self.n_patches ** 2 + 1, self.hidden_d)))
    self.pos_embed.requires_grad = False

  def forward(self, images):
    patches = patchify(images, self.n_patches)
    tokens = self.linear_mapper(patches)

    # Adding classification token to the tokens
    tokens = torch.stack([torch.vstack((self.class_token, tokens[i])) for i in range(len(tokens))])

    # Adding positional embedding
    pos_embed = self.pos_embed.repeat(images.shape[0], 1, 1)
    out = tokens + pos_embed

    return out

In [15]:
class MyViT(nn.Module):
    def __init__(self, chw=(1, 28, 28), n_patches=7, n_blocks=2, hidden_dim1=8, n_heads=2, out_d=10):
        # Super constructor
        super(MyViT, self).__init__()

        # Attributes
        self.chw = chw # (C, H, W)
        self.n_patches = n_patches
        self.hidden_d = hidden_dim1
        self.n_blocks = n_blocks
        self.n_heads = n_heads

        assert chw[1] % n_patches == 0, "Input shape not entirely divisible by number of patches"
        assert chw[2] % n_patches == 0, "Input shape not entirely divisible by number of patches"
        self.patch_size = (chw[1] / n_patches, chw[2] / n_patches)

        # 1) Linear mapper
        self.input_d = int(chw[0] * self.patch_size[0] * self.patch_size[1])
        self.linear_mapper = nn.Linear(self.input_d, self.hidden_d)

        # 2) Learnable classifiation token
        self.class_token = nn.Parameter(torch.rand(1, self.hidden_d)) # Classification Token을 정의

        # 3) Positional embedding
        self.pos_embed = nn.Parameter(torch.tensor(get_positional_embeddings(self.n_patches ** 2 + 1, self.hidden_d)))

        # 4) Transformer encoder blocks
        self.blocks = nn.ModuleList([MyViTBlock(self.hidden_d, n_heads) for _ in range(n_blocks)])


def forward(self, images):
        # Dividing images into patches
        n, c, h, w = images.shape
        patches = patchify(images, self.n_patches).to(self.positional_embeddings.device)

        # Running linear layer tokenization
        # Map the vector corresponding to each patch to the hidden size dimension
        tokens = self.linear_mapper(patches)

        # Adding classification token to the tokens
        tokens = torch.cat((self.class_token.expand(n, 1, -1), tokens), dim=1)

        # Adding positional embedding
        out = tokens + self.positional_embeddings.repeat(n, 1, 1)

        # Transformer Blocks
        for block in self.blocks:
            out = block(out)

        return out

최종적으로 Classificaion MLP까지 추가하면 ViT를 완성할 수 있습니다.

In [16]:
class MyViT(nn.Module):
    def __init__(
        self,
        chw=(1, 28, 28),
        n_patches=7,
        n_blocks=2,
        hidden_dim1=8,
        n_heads=2,
        out_d=10,
        hidden_d=None,           # 호환용: model(..., hidden_d=8)도 허용
    ):
        super(MyViT, self).__init__()

        if hidden_d is not None:
            hidden_dim1 = hidden_d

        self.chw = chw
        self.n_patches = n_patches
        self.hidden_d = hidden_dim1
        self.n_blocks = n_blocks
        self.n_heads = n_heads

        assert chw[1] % n_patches == 0 and chw[2] % n_patches == 0
        self.patch_size = (chw[1] // n_patches, chw[2] // n_patches)

        # 1) Linear mapper
        self.input_d = int(chw[0] * self.patch_size[0] * self.patch_size[1])  # 16
        self.linear_mapper = nn.Linear(self.input_d, self.hidden_d)

        # 2) Learnable classification token
        self.class_token = nn.Parameter(torch.randn(1, self.hidden_d) * 0.02)

        # 3) Positional embedding (freeze)
        self.pos_embed = nn.Parameter(
            get_positional_embeddings(self.n_patches ** 2 + 1, self.hidden_d)
        )
        self.pos_embed.requires_grad = False

        # 4) Transformer encoder blocks
        self.blocks = nn.ModuleList([MyViTBlock(self.hidden_d, n_heads) for _ in range(n_blocks)])

        # 5) Classification head (logits)
        self.mlp_head = nn.Linear(self.hidden_d, out_d)

    def forward(self, images):
        n, c, h, w = images.shape
        patches = patchify(images, self.n_patches)          # (N, 49, 16)
        tokens  = self.linear_mapper(patches)               # (N, 49, hidden_d)
        tokens  = torch.cat((self.class_token.expand(n, 1, -1), tokens), dim=1)  # (N, 50, hidden_d)

        out = tokens + self.pos_embed.repeat(n, 1, 1)       # (N, 50, hidden_d)

        for block in self.blocks:
            out = block(out)

        cls = out[:, 0, :]                                  # (N, hidden_d)
        logits = self.mlp_head(cls)                         # (N, out_d)
        return logits


## 4. Training

적절히 Epoch와 Learning Rate를 수정하여 loss를 최대한 낮춰보세요!

제출 조건: Test ACC를 75 이상 달성하기 :)

In [17]:
from tqdm import tqdm, trange

def main():
    # Loading data
    transform = ToTensor()

    train_set = MNIST(root='./datasets', train=True, download=True, transform=transform)
    test_set = MNIST(root='./datasets', train=False, download=True, transform=transform)

    train_loader = DataLoader(train_set, shuffle=True, batch_size=128)
    test_loader = DataLoader(test_set, shuffle=False, batch_size=128)

    # Defining model and training options
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Using device: ", device, f"({torch.cuda.get_device_name(device)})" if torch.cuda.is_available() else "")

    model = MyViT((1, 28, 28)).to(device)
    N_EPOCHS = 10
    LR = 1e-3

    # Training loop
    optimizer = Adam(model.parameters(), lr=LR)
    criterion = CrossEntropyLoss()

    for epoch in range(N_EPOCHS):
        train_loss = 0.0
        model.train()  # 모델을 학습 모드로 설정
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch + 1}/{N_EPOCHS} [Training]", leave=False)

        for batch in progress_bar:
            x, y = batch
            x, y = x.to(device), y.to(device)
            y_hat = model(x)
            loss = criterion(y_hat, y)

            train_loss += loss.detach().cpu().item() / len(train_loader)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # 현재 진행 상태의 loss를 tqdm에 업데이트
            progress_bar.set_postfix({"train_loss": train_loss})

        print(f"Epoch {epoch + 1}/{N_EPOCHS} train loss: {train_loss:.2f}")

        # Test loop
        model.eval()  # 모델을 평가 모드로 설정
        test_loss = 0.0
        correct, total = 0, 0
        test_progress_bar = tqdm(test_loader, desc=f"Epoch {epoch + 1}/{N_EPOCHS} [Testing]", leave=False)

        with torch.no_grad():
            for batch in test_progress_bar:
                x, y = batch
                x, y = x.to(device), y.to(device)
                y_hat = model(x)
                loss = criterion(y_hat, y)
                test_loss += loss.detach().cpu().item() / len(test_loader)

                correct += torch.sum(torch.argmax(y_hat, dim=1) == y).detach().cpu().item()
                total += len(x)

                # 현재 진행 상태의 test loss를 tqdm에 업데이트
                test_progress_bar.set_postfix({"test_loss": test_loss})

        accuracy = correct / total * 100
        print(f"Epoch {epoch + 1}/{N_EPOCHS} test loss: {test_loss:.2f}, test accuracy: {accuracy:.2f}%")


In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [19]:
model = MyViT((1, 28, 28), n_patches=7, n_blocks=2, hidden_d=8, n_heads=2, out_d=10).to(device)

In [20]:
if __name__ == "__main__":
    main()

100%|██████████| 9.91M/9.91M [00:00<00:00, 17.7MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 478kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.47MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 7.02MB/s]


Using device:  cuda (Tesla T4)


Epoch 1/10 train loss: 1.97


Epoch 1/10 test loss: 1.58, test accuracy: 39.24%


Epoch 2/10 train loss: 1.33


...

Epoch 8/10 test loss: 0.59, test accuracy: 80.67%


Epoch 9/10 train loss: 0.58


Epoch 9/10 test loss: 0.56, test accuracy: 81.44%


Epoch 10/10 train loss: 0.56


Epoch 10/10 test loss: 0.54, test accuracy: 82.27%
